Install required packages

In [6]:
#!pip install pandas scikit-learn sentence-transformers pyvis networkx xlsxwriter

sentence-transformers package needs torch, tensorflow or jax available as backend
I used torch

In [7]:
#!pip install torch

In [2]:
import pandas as pd

In [3]:
ISSUES_FILE = './all_issues.csv'
issue_data = pd.read_csv(ISSUES_FILE)
issue_data.head()

,Type,ID,Title,Description,Ethical Pillars,Requirements,Sub-requirements,WG,Unnamed: 8
0,Ethical Issue,E1,Small size of the training data set,The Brixia dataset might not be large enough t...,Prevention of harm,Technical Robustness and Safety,Fall-back plans and Reproducibility,technical,NaN
1,Ethical Issue,E1,Small size of the training data set,The Brixia dataset might not be large enough t...,Prevention of harm,Technical Robustness and Safety,Reliability,technical,NaN
2,Ethical Issue,E10,Low system transparency,It can be difficult to establish a link betwee...,Fairness,Accountability,Auditability,technical,NaN
3,Ethical Issue,E10,Low system transparency,It can be difficult to establish a link betwee...,Respect for Human Autonomy,Human Agency and Oversight,Human Oversight,technical,NaN
4,Ethical Issue,E10,Low system transparency,It can be difficult to establish a link betwee...,Prevention of harm,Technical Robustness and Safety,Resilience to attack and security,technical,NaN


In [4]:
issue_data["name"] = issue_data["ID"] + " - " + issue_data["WG"]
issue_data["text"] = issue_data["Title"] + " - " + issue_data["Description"]
issue_data.head()

,Type,ID,Title,Description,Ethical Pillars,Requirements,Sub-requirements,WG,Unnamed: 8,name,text
0,Ethical Issue,E1,Small size of the training data set,The Brixia dataset might not be large enough t...,Prevention of harm,Technical Robustness and Safety,Fall-back plans and Reproducibility,technical,NaN,E1 - technical,Small size of the training data set - The Brix...
1,Ethical Issue,E1,Small size of the training data set,The Brixia dataset might not be large enough t...,Prevention of harm,Technical Robustness and Safety,Reliability,technical,NaN,E1 - technical,Small size of the training data set - The Brix...
2,Ethical Issue,E10,Low system transparency,It can be difficult to establish a link betwee...,Fairness,Accountability,Auditability,technical,NaN,E10 - technical,Low system transparency - It can be difficult ...
3,Ethical Issue,E10,Low system transparency,It can be difficult to establish a link betwee...,Respect for Human Autonomy,Human Agency and Oversight,Human Oversight,technical,NaN,E10 - technical,Low system transparency - It can be difficult ...
4,Ethical Issue,E10,Low system transparency,It can be difficult to establish a link betwee...,Prevention of harm,Technical Robustness and Safety,Resilience to attack and security,technical,NaN,E10 - technical,Low system transparency - It can be difficult ...


In [5]:
issue_data_all = issue_data.copy()
issue_data.drop_duplicates(subset=['ID', 'WG'], inplace=True, ignore_index=True)
issue_data.head()

,Type,ID,Title,Description,Ethical Pillars,Requirements,Sub-requirements,WG,Unnamed: 8,name,text
0,Ethical Issue,E1,Small size of the training data set,The Brixia dataset might not be large enough t...,Prevention of harm,Technical Robustness and Safety,Fall-back plans and Reproducibility,technical,NaN,E1 - technical,Small size of the training data set - The Brix...
1,Ethical Issue,E10,Low system transparency,It can be difficult to establish a link betwee...,Fairness,Accountability,Auditability,technical,NaN,E10 - technical,Low system transparency - It can be difficult ...
2,Ethical Issue,E11,Sub-tasks might not need AI,The use of AI components for some sub tasks co...,Explicability,Transparency,Explainability,technical,NaN,E11 - technical,Sub-tasks might not need AI - The use of AI co...
3,Ethical Issue,E12,No feedback process in place,Radiologists using the AI tool do not have a s...,Fairness,"Diversity, non-discrimination and fairness",Stakeholder Participation.,technical,NaN,E12 - technical,No feedback process in place - Radiologists u...
4,Ethical Issue,E13,Explanations do not include regions of the lung.,the explanatory superpixels are created indepe...,Explicability,Transparency,Explainability,technical,NaN,E13 - technical,Explanations do not include regions of the lun...


In [6]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L12-v1')

In [7]:
issue_data["Embedding"] = issue_data["text"].apply(lambda x: model.encode(x))

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(issue_data["Embedding"].tolist())

In [136]:
import networkx as nx
top1_graph = nx.DiGraph()

top1_graph.add_nodes_from([
    (tup.Index, {
        'title': tup.text,
        'label': tup.name,
        'group': tup.WG,
        'size': 25
    })
    for tup in issue_data.itertuples()
    
])

for src in range(len(issue_data.index)):
    highest_similarity = np.argsort(-similarity[src,])[1:2] # self has highest similarity, find 5 others
    for dest in highest_similarity:
        weight_val = float(similarity[src,dest])
        top1_graph.add_edge(src, int(dest), weight=weight_val, value=weight_val, arrowStrikethrough=False)

rankings = nx.pagerank(nx.MultiGraph(top1_graph), weight='weight')
_min = min(rankings.values())
_max = max(rankings.values())

for node_id, rank in rankings.items():
    top1_graph.nodes[node_id]['importance'] = 15 + (rank - _min) / (_max - _min) * 10

In [137]:
from pyvis import network as net
nt = net.Network(notebook=True, height='900px', width='100%', directed=True)
nt.barnes_hut(spring_length=100, spring_strength=0.1, central_gravity=8, overlap=1)
nt.from_nx(top1_graph)
nt.show('test.html')

In [138]:
components_df = pd.concat([
    pd.DataFrame({
        'name': [top1_graph.nodes[node_id]['label'] for node_id in comp],
        'importance': [top1_graph.nodes[node_id]['importance'] for node_id in comp],
        'component': index
    })
    for index, comp in enumerate(sorted(nx.weakly_connected_components(top1_graph), key=len, reverse=True), start=1)
])
components_df.head()

,name,importance,component
0,E1 - Ethics,15.614044,1
1,E4 - Ethics,15.582459,1
2,E13 - technical,15.312210,1
3,F1 - technical,17.215740,1
4,F4 - technical,23.236499,1


In [143]:
issue_data_final = issue_data_all.merge(components_df)
issue_data_final.head()

,Type,ID,Title,Description,Ethical Pillars,Requirements,Sub-requirements,WG,Unnamed: 8,name,text,importance,component
41,Flag,F4,Influence on radiologists’ decisions is unclear,The current interaction between radiologist an...,Respect for Human Autonomy,Human Agency and Oversight,Human Agency,technical,NaN,F4 - technical,Influence on radiologists’ decisions is unclea...,23.236499,1
42,Flag,F4,Influence on radiologists’ decisions is unclear,The current interaction between radiologist an...,Respect for Human Autonomy,Human Agency and Oversight,Human Oversight,technical,NaN,F4 - technical,Influence on radiologists’ decisions is unclea...,23.236499,1
43,Flag,F4,Influence on radiologists’ decisions is unclear,The current interaction between radiologist an...,Prevention of harm,Technical Robustness and Safety,Accuracy,technical,NaN,F4 - technical,Influence on radiologists’ decisions is unclea...,23.236499,1
44,Flag,F4,Influence on radiologists’ decisions is unclear,The current interaction between radiologist an...,Prevention of harm,Technical Robustness and Safety,Reliability and Reproducibility,technical,NaN,F4 - technical,Influence on radiologists’ decisions is unclea...,23.236499,1
61,Flag,F2,Influence on doctors’ final decision is unclear,It is not clear to what extent the doctors tak...,Respect for human autonomy,Human agency and oversight,Human agency,social,NaN,F2 - social,Influence on doctors’ final decision is unclea...,18.776367,1


In [149]:
with pd.ExcelWriter('identified_clusters.xlsx') as writer:
    for component_number in sorted(issue_data_final['component'].unique()):
        component_data = (issue_data_final[issue_data_final['component'] == component_number]
                          .sort_values(['component', 'importance'], ascending=[True, False])
                          .drop(columns=['name', 'text', 'importance', 'component'])
                         )
        
        component_data.to_excel(writer, index=None, sheet_name=f"Cluster {component_number}")

In [18]:
top2_graph = net.Network(notebook=True, height='900px', width='100%', directed=True)
top2_graph.barnes_hut(spring_length=10, spring_strength=0.1, central_gravity=2, overlap=1)
# net.force_atlas_2based(spring_length=10)
top2_graph.add_nodes(
    issue_data.index, 
    title=issue_data['text'],
    label=issue_data['name'],
    color=issue_data['color']
)

for src in range(len(issue_data.index)):
    highest_similarity = np.argsort(-similarity[src,])[1:3] # self has highest similarity, find 5 others
    for dest in highest_similarity:
        top2_graph.add_edge(src, int(dest), value=float(similarity[src,dest]), arrowStrikethrough=False)

top2_graph.show("top-2.html")

In [20]:
queryId = "E1 - technical"
top_K = 5

index_row = issue_data[issue_data['name'] == queryId].index.tolist()[0]
similar_indices = np.argsort(-similarity[index_row,])[1:top_K+1]

issue_data.iloc[similar_indices].itertuples()
print(f"Query: {issue_data.loc[index_row].text}")
print("-----")
print("Most similar issues:")
for row in issue_data.iloc[similar_indices].itertuples():
        print(f"{row.name}: {row.text}")
        print(f"(Similarity: {similarity[index_row, row.Index]:.4f})")
        print("")

Query: Small size of the training data set - The Brixia dataset might not be large enough to allow for sufficient generalization to other populations. 
-----
Most similar issues:
E1 - healthcare: Lack of training data diversity - A major issue with the algorithm design lies in the dataset, primarily how representative the training data is of the population it will be used on in the future. The dataset should have geographic origin diversity as well as broad variation in past medical history, gender, age.
(Similarity: 0.5934)

E2 - technical: Data lacks representational fairness - The dataset used for training is likely not representative for the general population it is currently used on
(Similarity: 0.5625)

E6 - technical: No “hard” ground truth.  - Only a small part of the dataset is labeled through consensus of a group of experts, most of the data is labeled by a single radiologist. Furthermore the BrixiaScore (the label) is only semi-qualitative, so a label does not provide an obj